Code adopted from https://github.com/NVDLI/LDL. This code example is similar to autocomplete_rdo, but it works on words (encoded with an embedding layer) instead of characters and it does not do beam search. This version uses recurrent dropout and thus takes longer to train.

The initialization code below contains a couple of additional imports compared to c11_e1_autocomplete and defines two new constants MAX_WORDS and EMBEDDING_WIDTH that define the max size of our vocabulary and the dimensionality of the word vectors.


In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 32
BATCH_SIZE = 256
INPUT_FILE_NAME = './frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 3
MAX_WORDS = 7500
EMBEDDING_WIDTH = 100


2025-04-09 21:41:32.968603: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 21:41:32.976972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744252892.986609    2997 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744252892.989630    2997 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 21:41:33.000409: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

The next code snippet first reads the input file and splits the text into a list of individual words. The latter is done by using the imported function text_to_word_sequence(), which also removes punctuation and converts the text to lowercase. We then create input fragments and associated target words just as in the character-based example.


In [2]:
# Open and read file.
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8-sig')
text = file.read()
file.close()

# Make lower case and split into individual words.
text = text_to_word_sequence(text)

# Create training examples.
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])


The next step is to convert the training examples into the correct format. Each input word needs to be encoded to a corresponding word index (an integer). This index will then be converted into an embedding by the Embedding layer. The target (output) word should still be one-hot encoded. To simplify how to interpret the output, we want the one-hot encoding to be done in such a way that bit N is hot when the network outputs the word corresponding to index N in the input encoding.

We make use of the Keras Tokenizer class. When we construct our tokenizer, we provide an argument num_words = MAX_WORDS that caps the size of the vocabulary. The tokenizer object reserves index 0 to use as a special padding value and index 1 for unknown words. The remaining 9,998 indices (MAX_WORDS was set to 10,000) are used to represent words in the vocabulary.

The padding value (index 0) can be used to make all training examples within the same batch have the same length. The Embedding layer can be instructed to ignore this value, so the network does not train on the padding values.

Index 1 is reserved for UNKnown (UNK) words because we have declared UNK as an out-of-vocabulary (oov) token. When using the tokenizer to convert text to tokens, any word that is not in the vocabulary will be replaced by the word UNK. Similarly, if we try to convert an index that is not assigned to a word, the tokenizer will return UNK. If we do not set the oov_token parameter, it will simply ignore such words/indices.

After instantiating our tokenizer, we call fit_on_texts() with our entire text corpus, which will result in the tokenizer assigning indices to words. We can then use the function texts_to_sequences to convert a text string into a list of indices, where unknown words will be assigned the index 1.


In [3]:
# Convert to indices.
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
tokenizer.fit_on_texts(text)
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)

# Convert to appropriate input and output formats.
X = np.array(fragments_indexed, dtype=np.int64)
y = np.zeros((len(targets_indexed), MAX_WORDS))
for i, target_index in enumerate(targets_indexed):
    y[i, target_index] = 1


The next code snippet creates a model with an Embedding layer followed by two long short-term memory (LSTM) layers, followed by one fully connected layer with ReLU activation, and finally a fully connected layer with softmax as output. When we declare the Embedding layer, we provide it with its input dimensions (vocabulary size) and output dimensions (embedding width) and tell it to mask inputs using index 0. This masking is not necessary for our programming example given that we created the training input such that all input examples have the same length, but we do it for good practice. We state input_length=None so that we can feed training examples of any length to the network.


In [4]:
# Build and train model.
training_model = Sequential()
training_model.add(Input(shape=(None,), batch_size=BATCH_SIZE))
training_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True))
training_model.add(LSTM(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2))
training_model.add(LSTM(128, dropout=0.2,
                        recurrent_dropout=0.2))
training_model.add(Dense(128, activation='relu'))
training_model.add(Dense(MAX_WORDS, activation='softmax'))
training_model.compile(loss='categorical_crossentropy',
                       optimizer='adam')
training_model.summary()
history = training_model.fit(X, y, validation_split=0.05,
                             batch_size=BATCH_SIZE, 
                             epochs=EPOCHS, verbose=2, 
                             shuffle=True)


I0000 00:00:1744252895.409783    2997 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9209 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (256, None, 100)       │       750,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (256, None, 128)       │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (256, 128)             │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (256, 128)             │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (256, 7500)            │       967,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,982,844 (7.56 MB)

 Trainable params: 1,982,844 (7.56 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/32
98/98 - 70s - 716ms/step - loss: 7.0977 - val_loss: 7.9030
Epoch 2/32
98/98 - 67s - 680ms/step - loss: 6.4011 - val_loss: 8.1041
Epoch 3/32
98/98 - 67s - 686ms/step - loss: 6.2725 - val_loss: 8.2346
Epoch 4/32
98/98 - 68s - 692ms/step - loss: 6.1670 - val_loss: 8.2492
Epoch 5/32
98/98 - 67s - 688ms/step - loss: 6.0475 - val_loss: 8.4881
Epoch 6/32
98/98 - 67s - 682ms/step - loss: 5.9384 - val_loss: 8.4887
Epoch 7/32
98/98 - 67s - 683ms/step - loss: 5.8545 - val_loss: 8.6307
Epoch 8/32
98/98 - 66s - 677ms/step - loss: 5.7842 - val_loss: 8.7321
Epoch 9/32
98/98 - 66s - 677ms/step - loss: 5.7161 - val_loss: 8.7801
Epoch 10/32
98/98 - 70s - 718ms/step - loss: 5.6478 - val_loss: 8.9717
Epoch 11/32
98/98 - 66s - 676ms/step - loss: 5.5721 - val_loss: 8.7901
Epoch 12/32
98/98 - 66s - 669ms/step - loss: 5.4918 - val_loss: 8.9392
Epoch 13/32
98/98 - 67s - 681ms/step - loss: 5.4150 - val_loss: 9.0884
Epoch 14/32
98/98 - 68s - 698ms/step - loss: 5.3421 - val_loss: 9.1978
Epoch 15/32
98/

After training the model, we are ready to use it to do predictions. We do this a little bit differently than in the previous chapters. Instead of feeding a string of symbols as input to the model, we feed it only a single symbol at a time. This is an alternative implementation compared to the implementation in c11e1_autocomplete, where we repeatedly fed the model a growing sequence of characters (see the book for a more detailed description).

The scheme used in this chapter has a subtle implication, which has to do with dependencies between multiple consecutive calls to model.predict(). We want the LSTM layers to retain their c and h states from one call to another so that the outputs of subsequent calls to predict() will depend on the prior calls to predict(). This can be done by giving the parameter stateful=True to the LSTM layers. A side effect of this is that we manually need to call reset_states() on the model before our first prediction.

The code snippet below creates a model that is identical to the training model except that we declare the LSTM layers with stateful=True as well as specify a fixed batch size (required when declaring the LSTM layer as stateful) of size 1 using the batch_input_shape argument. We will transfer the weights from the trained model to this new model and use it for inference. This is done in the two last lines in the code snippet. There, we first read out the weights from the trained model and then initialize it into our inference model. For this to work, the models must have identical topology.


In [5]:
# Build stateful model used for prediction.
inference_model = Sequential()
inference_model.add(Input(shape=(None,), batch_size=1))
inference_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=False))
inference_model.add(LSTM(128, return_sequences=True,
                         dropout=0.2, recurrent_dropout=0.2,
                         stateful=True))
inference_model.add(LSTM(128, dropout=0.2,
                         recurrent_dropout=0.2, stateful=True))
inference_model.add(Dense(128, activation='relu'))
inference_model.add(Dense(MAX_WORDS, activation='softmax'))
weights = training_model.get_weights()
inference_model.set_weights(weights)


The next code snippet implements logic of presenting a word to the model and retrieving the word with the highest probability from the output. This word is then fed back as input to the model in the next timestep. The resulting autocompleted text sequence is printed.


In [6]:
# Provide beginning of sentence and
# predict next words in a greedy manner
first_words = ['i', 'saw']
first_words_indexed = tokenizer.texts_to_sequences(
    first_words)
inference_model.layers[1].reset_states()
inference_model.layers[2].reset_states()
predicted_string = ''
# Feed initial words to the model.
for i, word_index in enumerate(first_words_indexed):
    x = np.zeros((1, 1), dtype=np.int64)
    x[0][0] = word_index[0]
    predicted_string += first_words[i]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
# Predict PREDICT_LENGTH words.
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts(
        [[new_word_index]])
    x[0][0] = new_word_index
    predicted_string += word[0]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
print(predicted_string)


i saw the most creature 


All of the preceding code had to do with building and using a language model. The next code snippet adds some functionality to explore the learned embeddings. We first read out the word embeddings from the Embedding layer by calling get_weights() on layer 0, which represents the Embedding layer. We then declare a list of a number of arbitrary lookup words. This is followed by a loop that does one iteration per lookup word. The loop uses the Tokenizer to convert the lookup word to a word index, which is then used to retrieve the corresponding word embedding. The Tokenizer functions are generally assumed to work on lists. Therefore, although we work with a single word at a time, we need to provide it as a list of size 1, and then we need to retrieve element zero ([0]) from the output.

Once we have retrieved the corresponding word embedding, we loop through all the other embeddings and calculate the Euclidean distance to the embedding for the lookup word using the NumPy function norm(). We add the distance and the corresponding word to the dictionary word_indices. Once we have calculated the distance to each word, we simply sort the distances and retrieve the five word indices that correspond to the word embeddings that are closest in vector space. We use the Tokenizer to convert these indices back to words and print them and their corresponding distances.


In [7]:
# Explore embedding similarities.
embeddings = training_model.layers[0].get_weights()[0]
lookup_words = ['the', 'saw', 'see', 'of', 'and',
                'monster', 'frankenstein', 'read', 'eat']
for lookup_word in lookup_words:
    lookup_word_indexed = tokenizer.texts_to_sequences(
        [lookup_word])
    print('words close to:', lookup_word)
    lookup_embedding = embeddings[lookup_word_indexed[0]]
    word_indices = {}
    # Calculate distances.
    for i, embedding in enumerate(embeddings):
        distance = np.linalg.norm(
            embedding - lookup_embedding)
        word_indices[distance] = i
    # Print sorted by distance.
    for distance in sorted(word_indices.keys())[:5]:
        word_index = word_indices[distance]
        word = tokenizer.sequences_to_texts([[word_index]])[0]
        print(word + ': ', distance)
    print('')


words close to: the
the:  0.0
a:  3.8265364
“the:  4.5998383
an:  4.7560496
his:  5.2432733

words close to: saw
saw:  0.0
see:  0.47161064
seeing:  0.48724994
incomplete:  0.49550736
ban—as:  0.50558364

words close to: see
see:  0.0
saw:  0.47161064
distrust:  0.51923317
which:  0.52073133
sharing:  0.5220382

words close to: of
of:  0.0
in:  0.3527701
with:  0.37208176
by:  0.39471143
all:  0.40955588

words close to: and
and:  0.0
not:  0.37156314
deserved:  0.4141283
caroline:  0.42683053
populous:  0.42983887

words close to: monster
monster:  0.0
“fear:  0.4875126
wouldst:  0.49422055
nonproprietary:  0.49492013
ashes:  0.4983188

words close to: frankenstein
frankenstein:  0.0
redistribution:  0.44618255
efforts:  0.4595991
refund:  0.46755794
holder:  0.46883

words close to: read
read:  0.0
fierceness:  0.4762353
charge:  0.5105593
heart:  0.5150057
travel:  0.5172955

words close to: eat
eat:  0.0
forbear:  0.50898314
magnificent:  0.5213033
unhallowed:  0.52408516
private: 